# Importation des librairies python

In [13]:
import time
import re

# Création des différentes classes du logiciel

## Création de la classe interface_mail

#### La class interface_mail n’a qu’une seul méthode supplémentaire read_mail(), #### récupère le contenu du texte et cherche un motif commençant par le mot « salle » et alternant ensuite blocs de chiffres et boc d’autre caractères, censés être dans l’ordre : salle, date, horaire de début et horaire de fin. Le résultat est ensuite utilisé pour construire un objet reservation.On peut stocker les information ainsi extraite dans une liste pérenne qui sera ensuite traité en temps voulu par le reste du programme. Ainsi l’interface et la base de données peuvent travailler de façon asynchrone. C’est-à-dire que l’interface peut recevoir des mail en continu toute la journée et la base de données, être mise à jour une fois par jour.Pour la méthode read_mail(). Pour le moment on peut traiter les mails comme une chaine de caractère directement écrite dans le programme (Une interface utilisateur pourra être créer ultérieurement avec Flask). 

In [14]:
# lire le mail, recherche par motif REGEX (le mot salle), salle, date horraire debut/fi
# une fois traitement fais stockage dans une liste [] .append()

mail_test = "Salut, J’ai besoin de la salle 736 le 17 janvier entre 8 h et 10 h."

class interface_mail:

    def __init__(self):
        self.aTraiter = []
    def read_mail(self, mail):
        motif = "[a-zA-Z]+\s(\d+)[^\d]+([\d]+)\s*([a-zA-Z]+)[^\d]+([\d]+)[^\d]+([\d]+)"
        res_mail = re.findall(motif,mail)
        toAdd = [reservation(i) for i in res_mail]
        self.aTraiter += toAdd
    def __str__(self):
        testStr = ""
        #for resa in range(len(self.aTraiter)):
        for idx, resa in enumerate(self.aTraiter):
            testStr = "Numéro de salle: {}, Date de Resa: {}, Mois de Resa: {}, Heure de Début: {} , Heure de Fin: {}".format(resa.numSalle, resa.NumDateReserv, resa.MoisDateReserv, resa.heureDebut, resa.heureFin)
        return testStr

# Création de la classe réservation

## Pour la class reservation :
##### On peut supposer pour simplifier que les emails ont une structure ressemblant aux exemple cidessous, c’est-à-dire que chaque demande de la réservation se présente de la façon suivante : 
##### mention de la salle, puis de la date (jour avant mois), puis mention du créneau horaire.
##### Mail1.txt :
##### Bonjour,
##### Nous souhaiterions utiliser la salle 736 le mardi 17 janvier entre 15 h et 19 h et la salle 140 le 
##### 18 janvier 9 h – 11 h.
##### Mail2.txt :
##### Salut,
##### J’ai besoin de la salle 736 le 17 janvier entre 8 h et 10 h.
##### Mail3.txt :
##### Je peux avoir la salle 736 le mardi 17 janvier de 14 h à 16 h ?
##### Chris.
##### L’objet reservation doit pouvoir doit pouvoir contenir toutes ces informations, stockée sous forme de 
##### chaine de caractères, sauf les horaires qui seront des entiers pour pouvoir effectuer plus facilement 
##### des opérations

In [15]:
class reservation:
    def __init__(self, info):
        self.numSalle = info[0]
        self.NumDateReserv = info[1]
        self.MoisDateReserv = info[2]
        self.heureDebut = info[3]
        self.heureFin = info[4]

    def _set_salle(self, numSalle):
        self._numSalle = str(numSalle)
        
    def _set_date(self, NumDateReserv, MoisDateReserv):
        self._NumDateReserv = str(NumDateReserv)
        self._MoisDateReserv = str(MoisDateReserv)
        self._dateReserv = NumDateReserv + " " + MoisDateReserv
        
    
    def _set_heure(self, heureDebut,heureFin):
        self._heureDebut = int(heureDebut)
        self._heureFin = int(heureFin)
        self._Time = heureFin - heureDebut
    def _get_salle(self):
        return self._numSalle
    
    def _get_date(self):
        return self._dateReserv
    
    def _get_heure(self):
        return self._heureDebut, self._MoisDateReserv , self._Time
    
    numSalle = property(_get_salle, _set_salle)
    date = property(_get_date, _set_date)
    heure = property(_get_heure, _set_heure)

In [16]:
test = interface_mail()
test.read_mail(mail_test)
print(test)

Numéro de salle: 736, Date de Resa: 17, Mois de Resa: janvier, Heure de Début: 8 , Heure de Fin: 10


### Création de la classe Data access object

In [5]:
from sqlalchemy import create_engine
import pandas as pd
import json

fichierConfig = "config.json"
with open(fichierConfig) as fichier:
    config = json.load(fichier)["mysql"]

class DAO:


    def __init__(self):
        print("Welcome")
        engine = create_engine('mysql+' + config["connector"] + '://' + config["user"] + ":" + config["password"] + "@" + config["host"] + ":" + config["port"] + "/" + config["bdd"], echo=False)
        self.engine = engine

    def recuperer(self):
        query_load_reservation = pd.read_sql_query('SELECT res_date, res_heure_arrive, res_heure_depart , res_temps_reserver ,sa_name FROM RESERVATION JOIN SALLE on RESERVATION.sa_id = SALLE.sa_id;',engine)
        self.query_load_reservation = query_load_reservation
    
    def inserer(self):
        query_insert = pd.read_sql_query('SELECT * FROM SALLE',engine)
        self.query_insert = query_insert





#info_insert = DAO()
#DAO.inserer(info_insert)
#print(info_insert.query_insert)


In [6]:
info_load = DAO()
DAO.recuperer(info_load)
print(info_load.query_load_reservation)

Welcome


NameError: name 'engine' is not defined

### Création de la classe moteur

In [ ]:
class moteur: